### Extract and clean time series data

In [1]:
from utils import fetch_log_returns
training_dataset = fetch_log_returns(start = '1995-01-01', end ='1995-12-31')

[*********************100%***********************]  5 of 5 completed


In [2]:
training_dataset.head()

,sp500,dax,ftse,nikkei,ibex
Date,,,,,
1995-01-05 00:00:00+00:00,-0.000803,-0.010397,-0.006345,-0.003457,-0.019565
1995-01-06 00:00:00+00:00,0.000738,0.003280,0.010726,-0.004939,0.000000
1995-01-09 00:00:00+00:00,0.000326,-0.002330,-0.003006,-0.003826,-0.014346
1995-01-10 00:00:00+00:00,0.001843,0.004213,0.001504,0.002903,-0.005274
1995-01-11 00:00:00+00:00,-0.000043,-0.000155,-0.003601,0.002408,-0.010125


### Define the problem instance and model parameters

In [3]:
from model_params import LAMBDA_1, LAMBDA_2, LAMBDA_3, NLAYERS, NSHOTS, NUM_ASSETS, SIGMA_TARGET, TWO_QUBIT_GATES, K, N, RISK_FREE_RATE


### Build the circuit Ansatz (without angle embedding)

In [4]:
from ansatz import build_hardware_efficient_ansatz
from ansatz import compute_number_of_params
ansatz = build_hardware_efficient_ansatz(num_qubits=N, num_layers=NLAYERS, params=[0]*compute_number_of_params(N,NLAYERS))
print('Hardware efficient circuit ansatz')
print()
print(ansatz.draw())


Hardware efficient circuit ansatz

q0: ─U2─U2─o─────────────────U1─U2─o─────────────────U1─M─
q1: ─U2─U2─X─o───────────────U1─U2─X─o───────────────U1─M─
q2: ─U2─U2───X─o─────────────U1─U2───X─o─────────────U1─M─
q3: ─U2─U2─────X─o───────────U1─U2─────X─o───────────U1─M─
q4: ─U2─U2───────X─o─────────U1─U2───────X─o─────────U1─M─
q5: ─U2─U2─────────X─o───────U1─U2─────────X─o───────U1─M─
q6: ─U2─U2───────────X─o─────U1─U2───────────X─o─────U1─M─
q7: ─U2─U2─────────────X─o───U1─U2─────────────X─o───U1─M─
q8: ─U2─U2───────────────X─o─U1─U2───────────────X─o─U1─M─
q9: ─U2─U2─────────────────X─U1─U2─────────────────X─U1─M─


### Build the Portfolio Optimization Hamiltonian and run the optimization process


In [5]:
import numpy as np
from qibo.optimizers import optimize
from cost_function import compute_total_energy

# Optimize starting from a random guess for the variational parameters
initial_params = np.random.uniform(0, 2*np.pi, compute_number_of_params(N,NLAYERS))

# perform optimization
best, optimal_params, extra = optimize(compute_total_energy, initial_params, args=(training_dataset))

# set final solution to circuit instance
print('Ground state energy:', best)
print()
print('Optimal parameters', optimal_params)
print()
print('Optimization process info', extra)

[Qibo 0.2.8|INFO|2024-10-03 21:15:35]: Using numpy backend on /CPU:0


Ground state energy: 506.12950109202353

Optimal parameters [3.00305892 6.02305148 2.69511672 4.16214273 2.90799832 3.73777956
 3.80217565 1.70943149 1.60381381 0.28194164 3.38269684 1.78528101
 0.80137024 1.31614047 0.98534239 1.34510744 0.99719482 0.4242799
 1.6479977  4.8909712  5.98982794 3.64727171 6.01979223 3.80448353
 1.70600622 0.29706119 3.49400537 0.06915121 1.00814346 2.17900223
 3.67949402 3.97694322 3.43694895 3.86917298 3.67168101 3.75558203
 6.20371015 3.70428588 3.06427133 4.2945571  4.45276059 3.03980572
 3.26606774 3.81923083 1.1652326  3.48640188 6.23719505 5.90565563
 3.63053771 5.86849003 0.0468147  4.6658484  2.73274717 5.47468807
 3.36715464 2.33688017 0.84258889 5.9772209  5.76589526 3.32288782
 1.53086988 4.87706024 4.00721624 2.65588922 0.88979672 4.73602043
 2.3164678  3.92605223 0.69398223 2.99060068 0.57517151 0.78435133
 4.02614136 1.91175548 5.6446198  3.17190536 0.6703977  2.11970854
 5.34155657 2.72978963]

Optimization process info  message: Optimizat

### Results parsing


In [7]:
def get_minimum_energy(portfolios: dict) -> float:
    energies = []
    for data in portfolios.values():
        energies.append(data['energy'])
    return min(energies)


In [8]:
from qibo.result import CircuitResult
def get_max_prob(result: CircuitResult, nshots: int = NSHOTS) -> float:
    number_of_times = result.frequencies().values()
    probs = [freq/nshots for freq in number_of_times]
    return max(probs)

In [9]:
result = ansatz(nshots=100)


In [10]:
from qibo.models import Circuit
from cost_function import compute_cost_function
from utils import string_to_int_list
import pandas as pd
TOLERANCE = 0.5

def get_optimal_binary_portfolios(ansatz: Circuit, dataset: pd.DataFrame, nshots: int = NSHOTS, tolerance: int = TOLERANCE) -> dict:
    result = ansatz(nshots=nshots)
    optimal_portfolios = {}
    for bit_string, stat_freq in result.frequencies().items():
        if (get_max_prob(result) - stat_freq/nshots) < tolerance:
            optimal_portfolios[bit_string] = {'stat_freq': stat_freq/nshots, 'energy': compute_cost_function(dataset, string_to_int_list(bit_string))}
    return optimal_portfolios


optimized_ansatz = ansatz.set_parameters(optimal_params)
optimal_binary_portfolios = get_optimal_binary_portfolios(ansatz, training_dataset)


In [11]:
optimal_binary_portfolios

{'0000000101': {'stat_freq': 0.01, 'energy': 563.8917639195123},
 '0000100110': {'stat_freq': 0.01, 'energy': 563.8960217337241},
 '0000101010': {'stat_freq': 0.01, 'energy': 624.8323520703171},
 '0000110000': {'stat_freq': 0.02, 'energy': 624.8591555331126},
 '0000110111': {'stat_freq': 0.01, 'energy': 351.4824031395772},
 '0000111110': {'stat_freq': 0.01, 'energy': 399.8821337830648},
 '0001011010': {'stat_freq': 0.01, 'energy': 451.4402725211241},
 '0001100010': {'stat_freq': 0.01, 'energy': 563.9153748152622},
 '0001101111': {'stat_freq': 0.01, 'energy': 306.1453188734975},
 '0001110011': {'stat_freq': 0.01, 'energy': 351.5017562211234},
 '0001110110': {'stat_freq': 0.01, 'energy': 351.46516443776204},
 '0001111101': {'stat_freq': 0.01, 'energy': 263.9857982565288},
 '0010001010': {'stat_freq': 0.01, 'energy': 624.8194763269429},
 '0010001011': {'stat_freq': 0.01, 'energy': 506.1084113929029},
 '0010010000': {'stat_freq': 0.01, 'energy': 624.8462834598382},
 '0010011110': {'stat_fr

In [12]:
def get_asset_weights_binary(assets, ordered_bitstring, num_qubit_per_asset = K):
    weights = [ordered_bitstring[i:i+num_qubit_per_asset] for i in range(0, len(ordered_bitstring),num_qubit_per_asset)]
    return dict(zip(assets,weights))

optimal_binary_portfolio=list(optimal_binary_portfolios.keys())[-2]

binary_asset_weights = get_asset_weights_binary(training_dataset.columns, optimal_binary_portfolio)
binary_asset_weights

{'sp500': '11', 'dax': '11', 'ftse': '11', 'nikkei': '00', 'ibex': '10'}

In [13]:

def get_asset_weight_decimal(asset_bit_string):
    w = 0
    for k,bit in enumerate(asset_bit_string):
       w += 2**(k-1)*int(bit)*1/(2**len(asset_bit_string)) # discretization !
    return w  

get_asset_weight_decimal(binary_asset_weights['nikkei'])

0.0

In [14]:
portfolio = {}
for asset, w in binary_asset_weights.items():
    portfolio[asset] = get_asset_weight_decimal(w)
portfolio

{'sp500': 0.375, 'dax': 0.375, 'ftse': 0.375, 'nikkei': 0.0, 'ibex': 0.125}

### Calculating portfolio metrics

In [17]:
def get_portfolio_metrics(portfolio: dict, dataset: pd.DataFrame, r: float = RISK_FREE_RATE):
    import numpy as np
    normalized_weights = list(portfolio.values()) / np.sum(list(portfolio.values()))


    # Calculate the expected log returns, and add them to the `returns_array`.
    annualized_ret_portfolio = np.sum((dataset.mean() * normalized_weights) * 252)


    # Calculate the volatility, and add them to the `volatility_array`.
    annualized_vol_portfolio = np.sqrt(
        np.dot(normalized_weights.T, np.dot(dataset.cov() * 252, normalized_weights))
    )
    annualized_ret_portfolio,annualized_vol_portfolio

    # Calculate the Sharpe Ratio and Add it to the `sharpe_ratio_array`.
    sharpe_ratio= (annualized_ret_portfolio-r)/annualized_vol_portfolio

    # Let's create our "Master Data Frame", with the weights, the returns, the volatility, and the Sharpe Ratio
    return {'Returns':annualized_ret_portfolio, 'Volatility':annualized_vol_portfolio, 'Sharpe Ratio':sharpe_ratio, 'Normalized Weights':normalized_weights}
get_portfolio_metrics(portfolio, training_dataset)

{'Returns': 0.18201864332913661,
 'Volatility': 0.0832295963967742,
 'Sharpe Ratio': 1.8264974229170783,
 'Normalized Weights': array([0.3, 0.3, 0.3, 0. , 0.1])}

Parte fácil NO guiada: 
- Resolver por fuerza bruta para ver lo mala que es la optimización pero lo precisa que es la formulación

Para hacer por libre si sabes mucho: 

- Trainable -> unsupervised quantum machine learning
